# Comparing ground truth with Prince

In this notebook, we compare the width computed with the high magnification microscope (x100) to those taken with Prince. 

## Setup

In [1]:
from amftrack.util.sys import data_path, get_current_folders, update_plate_info
from amftrack.util.file import chose_file
from amftrack.util.plot import show_image
from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
import os
import matplotlib.pyplot as plt
import logging
#%matplotlib notebook

/home/felix/Wks/AMFtrack/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
directory = "/data/felix/width1/full_plates/"
plate_name = "20220325_1423_Plate907"
update_plate_info(directory)
folder_df = get_current_folders(directory)

FileNotFoundError: [Errno 2] No such file or directory: '/data/felix/width1/full_plates/'

In [ ]:
selected_df = folder_df.loc[folder_df['folder']==plate_name]

In [14]:
i = 0

In [15]:
plate = int(list(selected_df['folder'])[i].split('_')[-1][5:])
folder_list = list(selected_df['folder'])
directory_name = folder_list[i]

In [16]:
exp = Experiment(plate, directory)
exp.load(selected_df.loc[selected_df['folder']==directory_name],labeled=False)

2022-03-25 14:23:00


In [17]:
len(exp.nx_graph[0])

2407